# Deploying Models

Let's get started by installing some packages (skip if you have them).

In [ ]:
!pip install boto3
!pip install sagemaker

In [3]:
import json
import boto3
import sagemaker as sage
from sagemaker import get_execution_role
from sagemaker import ModelPackage

Let's first create the configuration for each of the models that we are going use.

In [4]:
SUMMARIZER_MODEL_CONFIG = {
    'name': 'summarizer',
    'content_type':'text/plain',
    'instance_type': 'ml.m4.xlarge',
    'arn': 'arn:aws:sagemaker:us-east-1:865070037744:model-package/marketplace-text-summarizer-e316c0bc7da2305368d5493465af0802'
}
EXPERT_TICKETS_MODEL_CONFIG = {
    'name': 'expert-tickets',
    'content_type':'text/csv',
    'instance_type': 'ml.m5.xlarge',
    'arn': 'arn:aws:sagemaker:us-east-1:865070037744:model-package/marketplace-expert-b3bae5926e152cd3c0e2e783cc50f837'
}
ADDRESS_MODEL_CONFIG = {
    'name': 'address-extract',
    'content_type':'text/plain',
    'instance_type': 'ml.m4.xlarge',
    'arn': 'arn:aws:sagemaker:us-east-1:865070037744:model-package/address-extraction-update-copy-2842b75335671181216be9b46aec2af6'
}
WIREFRAME_CONFIG = {
    'name': 'wireframe',
    'content_type':'image/jpeg',
    'instance_type': 'ml.m5.xlarge',
    'arn': 'arn:aws:sagemaker:us-east-1:865070037744:model-package/wireframetocode-01-30-517aa954c6cea79683836daa20955a5c'
}
SIMILARITY_CONFIG = {
    'name': 'similarity',
    'content_type':'ml.m5.4xlarge',
    'instance_type': 'ml.m5.4xlarge',
    'arn': 'arn:aws:sagemaker:us-east-1:865070037744:model-package/dynamic-ai-demo-617ce7e562c6bd730e798be57baa4605'
}

PASSPORT_CONFIG = {
    'name': 'passport',
    'content_type':'image/jpeg',
    'instance_type': 'ml.m4.xlarge',
    'arn': 'arn:aws:sagemaker:us-east-1:865070037744:model-package/gtriip-passport-ic-m5-7-62ae9e20b9779c8073d3393d0d93b5fe'
}

Here we create a generic class called **Model** to create a model based on its configuration.

In [5]:
class Model():
    def __init__(self, config):
        self.name = config['name']
        self.instance_type =  config['instance_type']
        self.content_type =  config['content_type']
        self.model_package_arn =  config['arn']
    
    def _predict_wrapper(self, endpoint, session):
        return sage.RealTimePredictor(endpoint, session, self.content_type)
    
    def cleanup(self):
        self.predictor.delete_endpoint(delete_endpoint_config=True)
        self.predictor.delete_model()        
        
    def remove_endpoint(self):
        self.predictor.delete_endpoint(delete_endpoint_config=True)

    def create_and_deploy(self, role, session):
        #create a deployable model from the model package.
        self.model_package = ModelPackage(role=role,
                            model_package_arn=self.model_package_arn,
                            sagemaker_session=session,
                            predictor_cls=self._predict_wrapper)

        #Deploy the model
        self.predictor = self.model_package.deploy(1, self.instance_type, endpoint_name=self.name)
        return self.predictor

Let's create a session.

In [6]:
session = sage.Session()
role = 'AmazonSageMakerFull'

### Deploy passport model

In [ ]:
model_passport = Model(PASSPORT_CONFIG)
model_passport.create_and_deploy(role, session)

### Deploy wireframe model

In [ ]:
model_wireframe = Model(WIREFRAME_CONFIG)
model_wireframe.create_and_deploy(role, session)

### Deploy summarizer model

In [ ]:
model_summarizer = Model(SUMMARIZER_MODEL_CONFIG)
model_summarizer.create_and_deploy(role, session)

### Deploy address extractor model

In [ ]:
model_address = Model(ADDRESS_MODEL_CONFIG)
model_address.create_and_deploy(role, session)

### Deploy expert ticket suggestion model

In [ ]:
model_experts = Model(EXPERT_TICKETS_MODEL_CONFIG)
model_experts.create_and_deploy(role, session)

# Inference

First, let's define the class to simplify the invocation of the models

In [7]:
class ModelEndpoint():
    
    def __init__(self, config, runtime):           
        self.content_type = config['content_type']
        self.endpoint = config['name']
        self.runtime = runtime
        
    def predict(self, params):          
        # Send data via InvokeEndpoint API
        response = self.runtime.invoke_endpoint(EndpointName=self.endpoint, ContentType=self.content_type, Body=params)
        # Unpack response
        result = response['Body'].read().decode()
        return result   


Now let's create the endpoint reference for each model

In [8]:
runtime = boto3.Session().client('sagemaker-runtime')

address_model_endpoint = ModelEndpoint(ADDRESS_MODEL_CONFIG, runtime)
wireframe_model_endpoint = ModelEndpoint(WIREFRAME_CONFIG, runtime)
expert_tickets_model_endpoint = ModelEndpoint(EXPERT_TICKETS_MODEL_CONFIG, runtime)
summarizer_model_endpoint = ModelEndpoint(SUMMARIZER_MODEL_CONFIG, runtime) 
similarity_model_endpoint = ModelEndpoint(SIMILARITY_CONFIG, runtime)
passport_model_endpoint = ModelEndpoint(PASSPORT_CONFIG, runtime)


## Inference on Mphasis DeepInsights Address Extraction


In [31]:
address_model_endpoint.predict('RKH Specialty Page 3 of 4 UMR: B0180 ME1706357 Fiscal and Regulatory Section TAX PAYABLE BY INSURER(S): None applicable. COUNTRY OF ORIGIN: United States of America. OVERSEAS BROKER: 331 E 132nd St Mott Haven, USA')

',Addresses\n0,OVERSEAS BROKER 331 E 132nd St Mott Haven USA\n'

## Inference on Mphasis Autocode WireframeToCode


In [36]:
with open('./data-input/wireframe.jpg','rb') as f:
    image = f.read()
    
result = wireframe_model_endpoint.predict(image)
result_json = json.loads(result)

print (result_json)

{'generated_webpage_html': '<!DOCTYPE html>\n\n<html lang="en">\n\n<head>\n\n  <title>Bootstrap Example</title>\n  <meta charset="utf-8">\n  <meta name="viewport" content="width=device-width, initial-scale=1">\n  <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/4.1.0/css/bootstrap.min.css">\n  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-awesome.min.css">\n  <link href="https://fonts.googleapis.com/icon?family=Material+Icons" rel="stylesheet">\n  <script src="https://maxcdn.bootstrapcdn.com/bootstrap/4.1.0/js/bootstrap.min.js"></script>\n  <link rel="stylesheet" href="autocodeai-form.css">\n\n</head>\n\n<body>\n\n<header>\n\n</header>\n\n<main class="main">\n\n<form>\n\n\n\n            <div style="position:absolute;top:128px;left:112px;">\n            <label for="comment" style="width:288px;height:48px">Label</label>\n            </div>\n            \n                <div style="position:absolute;top:192px;left:11

## Inference on Mphasis Optimize.AI Expert Identifier


In [9]:
input_csv = open('./data-input/experts-tickets.csv').read()
result_expert = expert_tickets_model_endpoint.predict(input_csv)

print (result_expert)

,Request ID,Request Submitted Date and Time,Request Priority,Request Category,Request Status,Request Resolved By
0,45098,2019-02-25,3-Low,IT Department,Open,Paul



## Inference on Mphasis DeepInsights Text Summarizer


In [41]:
text_privacy = open('./data-input/Policy Document.txt').read()
summarizer_model_endpoint.predict(text_privacy)

'Eu-us and swiss-us privacy shield what personal information about customers does amazon europe collect ?we collect your personal information in order to provide and continually improve our products and services. Automatic information : we automatically receive and store certain types of information when you use amazon services , such as information about your use , including your interaction with content and services available through amazon services.\nExecution time : 2.46seconds\n'

## Inference on Passport Data Page Detection

In [45]:
with open('./data-input/passport.jpg','rb') as f:
    content = f.read()

response = passport_model_endpoint.predict(content)

print(json.dumps(json.loads(response), indent=4, sort_keys=True))

{
    "data": {
        "check_composite": "2",
        "check_date_of_birth": "8",
        "check_expiration_date": "2",
        "check_number": "2",
        "check_personal_number": "0",
        "country": "United Kingdom of Great Britain and Northern Ireland",
        "date_of_birth": "17/01/1985",
        "expiration_date": "31/01/2016",
        "expired": "true",
        "names": "GUSTAVO ZMR",
        "nationality": "GBR",
        "number": "107185703",
        "personal_number": "",
        "sex": "Male",
        "surname": "UNITED KINGDOH FIVE",
        "valid_composite": true,
        "valid_date_of_birth": true,
        "valid_expiration_date": true,
        "valid_number": true,
        "valid_personal_number": true,
        "valid_score": 100
    },
    "reason": "",
    "valid": true
}


# Remove models and endpoints

In [ ]:
model_passport.cleanup()
model_wireframe.cleanup()
model_summarizer.cleanup()
model_address.cleanup()
model_experts.cleanup()